In [1]:
%matplotlib inline

In [2]:
import os
import netCDF4
import numpy as np
from geophys_utils import NetCDFLineUtils
import matplotlib.pyplot as plt

In [3]:
# Setup proxy as required
GA_STAFF_WIFI = False

if GA_STAFF_WIFI:
    os.environ['http_proxy'] = 'http://proxy.inno.lan:3128'
    os.environ['https_proxy'] = 'http://proxy.inno.lan:3128'

In [4]:
nc_path = "/iv65/Geoscience_Australia_Geophysics_Reference_Data_Collection/airborne_geophysics/SA/line/P1255/P1255-line-elevation-Marree.nc"
if not os.path.isfile(nc_path):
    nc_path = "https://dapds00.nci.org.au//thredds/dodsC/iv65/Geoscience_Australia_Geophysics_Reference_Data_Collection/airborne_geophysics/SA/line/P1255/P1255-line-elevation-Marree.nc"

bounds = (137, -29, 138, -28) # Spatial subset of dataset

In [5]:
# Initialise netCDF dataset object
nc_dataset = netCDF4.Dataset(nc_path + '#fillmismatch') # Note work-around for bad _FillValue: https://github.com/Unidata/netcdf-c/issues/1299

In [8]:
# Initialise NetCDFLineUtils object
# Code is at https://github.com/geoscienceaustralia/geophys_utils
netcdf_line_utils = NetCDFLineUtils(nc_dataset, debug=True)
#netcdf_line_utils.__dict__

In [9]:
# Test get_spatial_mask function and count the points within the bounding box
spatial_mask = netcdf_line_utils.get_spatial_mask(bounds)
np.count_nonzero(spatial_mask)

AttributeError: 'NoneType' object has no attribute 'ExportToWkt'

In [ ]:
# Count the points in a couple of lines
for line_number, line_mask in netcdf_line_utils.get_line_masks(line_numbers=[100060, 100070]):
    print('Line {} has {} points'.format(line_number, np.count_nonzero(line_mask)))

In [ ]:
# Get all coordinates and data points for line 100060
line_number, line_dict = next(netcdf_line_utils.get_lines(line_numbers=100060))
#print(line_number, line_dict)

In [ ]:
# Convert coordinates to UTM and compute cumulative distances in metres for line 100060
utm_distances = netcdf_line_utils.coords2metres(line_dict['coordinates'])
print(utm_distances)

In [ ]:
# Plot data points against line distance for line 100060
plt.figure(figsize=(30,20))   
for varname in line_dict.keys():
    if varname != 'coordinates':
        print(varname)
        plt.plot(utm_distances, line_dict[varname])

In [ ]:
# Grid points to 100m resolution in UTM
utm_mag_awags_grid, crs, geotransform = netcdf_line_utils.utm_grid_points(utm_grid_resolution=100.0, 
                                               variables='mag_awags', 
                                               native_grid_bounds=bounds,
                                               point_step=1)
print(crs)
print(geotransform)
print(utm_mag_awags_grid.shape)

In [ ]:
plt.figure(figsize=(30,20))    
plt.imshow(utm_mag_awags_grid)